# Creating a new user

The Small Business Association (SBA) database (sba) is now live after many weeks of design and development effort. Your supervisor has asked you to create a new user account for this database to be used by a new hire, Sandra Gold, on the software development team.

```
-- Create sgold with a temporary password
CREATE USER sgold WITH PASSWORD 'changeme';
```

# Updating user passwords

Sandra Gold has just started her new job at the SBA. She is excited to get started in this new role. On her first day, her on-boarding process includes receiving access to the SBA database. She is instructed to access her account using the temporary password that she was given and change her password to be more secure. You will have the opportunity to assume the role of Sandra and update the password for her account.

```
-- Update the password for sgold
ALTER USER sgold WITH PASSWORD 'kxqr478-?egH%&FQ';
```

# Granting user privileges

Sandra Gold's first development project is the creation of an online platform for processing loans. This platform will be used to perform actions such as registering new loan applications and updating loan terms. Recall that loan data is stored in the loan table of the sba database. Your user account can grant privileges to tables in the sba database. You will grant privileges that Sandra Gold requires to develop the platform for managing loan processing.

```
-- Grant the INSERT privilege
GRANT INSERT ON loan TO sgold;

-- Grant the UPDATE privilege
GRANT UPDATE ON loan TO sgold;

-- Grant the SELECT privilege
GRANT SELECT ON loan TO sgold;

-- Grant the DELETE privilege
GRANT DELETE ON loan TO sgold;
```

# Using the granted privileges

Now that Sandra has been granted the INSERT, UPDATE, SELECT, and DELETE privileges, she has been able to use the loan table. She successfully manages what data is in the table. However, she is working on an aspect of her project that requires an alteration to the loan table. Specifically, she would like to start capturing the approval date AND time of a loan. This requires altering the structure of the table using the following commands:
```
ALTER TABLE loan DROP COLUMN approval_date;
ALTER TABLE loan ADD COLUMN approval_dt DATETIME;
```
Now you will change ownership of the loan table so that Sandra can begin capturing both the approval date and approval time.

```
-- Provide sgold with the required table privileges
ALTER TABLE loan OWNER TO sgold;
```

# Working with users and groups

Sandra's project at the SBA is turning into a great success. Loan processing is now much faster than before the rollout of the online loan platform. This results in the growth of the team responsible for the loan management platform. Sandra is now leading the team and recently hired 3 new developers. Before the new developers begin their first days on the job, Sandra would like to create database accounts for each user and give each account the same access privileges.

```
-- Create a user account for Ronald Jones
CREATE USER rjones WITH PASSWORD 'changeme';

-- Create a user account for Kim Lopez
CREATE USER klopez WITH PASSWORD 'changeme';

-- Create a user account for Jessica Chen
CREATE USER jchen WITH PASSWORD 'changeme';

-- Create the dev_team group
CREATE GROUP dev_team;

-- Grant privileges to dev_team group on loan table
GRANT INSERT, UPDATE, DELETE, SELECT ON loan TO dev_team;

-- Add the new user accounts to the dev_team group
ALTER GROUP dev_team ADD USER rjones, klopez, jchen;
```

# Schema privileges

The new software development team members are eager to get started on the loan management project. But Sandra, as the team lead, is not comfortable with so many people having direct access to the production version of the loan database (in the public schema). The rapidly growing team must implement some software development best practices which include setting up a development environment that is separated from the production environment. In this exercise, you will create a new schema to represent this development environment and grant members of the software development team privileges on this schema.

```
-- Create the development schema
CREATE SCHEMA development;

-- Grant usage privilege on new schema to dev_team
GRANT USAGE ON SCHEMA development TO dev_team;

-- Create a loan table in the development schema
CREATE TABLE development.loan (
	borrower_id INTEGER,
	bank_id INTEGER,
	approval_date DATE,
	program text NOT NULL,
	max_amount DECIMAL(9,2) NOT NULL,
	gross_approval DECIMAL(9, 2) NOT NULL,
	term_in_months SMALLINT NOT NULL,
	revolver_status BOOLEAN NOT NULL,
	bank_zip VARCHAR(10) NOT NULL,
	initial_interest_rate DECIMAL(4, 2) NOT NULL
);

-- Grant privileges on development schema
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA development TO dev_team;
```

# Removing user privileges

It seems that success on the loan platform team is contagious. One of the team members, Kim Lopez, has been promoted and will be taking on a project management role. She won't need to make any modifications to the loan data going forward. However, having the ability to query the loan table will still be useful in her new role.

```
-- Remove the specified privileges for Kim
REVOKE INSERT, UPDATE, DELETE ON development.loan FROM klopez;
```

# Rescinding group membership

The technology team at the SBA would like to be more proactive in assigning database access by group. Each user account must belong to at least one group. All project managers will be members of a project_management group. Therefore, Kim Lopez, should be added to this account and removed from the dev_team group. This exercise will allow you to manage group membership for these teams.

```
-- Create the project_management group
CREATE GROUP  project_management;

-- Grant project_management SELECT privilege
GRANT SELECT ON loan TO project_management;

-- Add Kim's user to project_management group
ALTER GROUP project_management ADD USER klopez;

-- Remove Kim's user from dev_team group
REVOKE dev_team FROM klopez;
```

# Implementing access control for teams

You will create a new schema and table for holding data that will be provided to data scientists on your team. While your team members are likely responsible individuals, accidents can happen. You should only give these team members as much control over the database as required to do their job. These team members will have access to data on loans that have not been approved.

- The schema analysis will be created.
- The table unapproved_loan will be defined in this new schema.
- User data_scientist will be created.
- The user will be restricted to reading from the new table.

```
-- Create the new analysis schema
CREATE SCHEMA analysis;

-- Create a table unapproved loan under the analysis schema
CREATE TABLE analysis.unapproved_loan (
    id serial PRIMARY KEY,
    loan_id INTEGER REFERENCES loan(id),
    description TEXT NOT NULL
);

-- Create 'data_scientist' user with password 'changeme'
CREATE USER data_scientist WITH PASSWORD 'changeme';

-- Give 'data_scientist' ability to use 'analysis' schema
GRANT USAGE ON SCHEMA analysis TO data_scientist;

-- Grant read-only access to table for 'data_scientist' user
GRANT SELECT ON analysis.unapproved_loan TO data_scientist;
```